In [1]:
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import torch
import os
from tqdm import tqdm
from PIL import Image
import random
import tensorflow as tf
import torchvision.transforms.functional as TransformerF
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import transforms, models
import torch.nn.functional as F
import math
from copy import deepcopy
import json
import time



2025-07-01 05:43:02.755119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751348582.778083     235 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751348582.785308     235 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class ImageCropper():
  def __init__(self,debuging,target_size = (512, 512) ):
    self.target_size=target_size
    self.margin=20
    self.debuging= debuging
    self.transform=None
    self.normalized=False
    if debuging:
      self.cropped_image = None

  def cropImage(self,image, landmarks, checker):
    if torch.is_tensor(image):
        image = image.cpu().numpy()
        image = (image).astype(np.uint8)
    if torch.is_tensor(landmarks):
        landmarks_np = landmarks.cpu().numpy()
    else:
        landmarks_np = np.array(landmarks).copy()

    # Ensure landmarks shape is (N, 2)
    if landmarks_np.ndim == 1 and landmarks_np.shape[0] % 2 == 0:
        landmarks_np = landmarks_np.reshape(-1, 2)
    elif landmarks_np.ndim != 2 or landmarks_np.shape[1] != 2:
        raise ValueError(f"Invalid landmarks shape: {landmarks_np.shape}")
    
    frame = image
    lm = landmarks_np
    if not checker['face_landmarks']:
        variable = lm[6+21]
        variable[1]-=50
        lm[54:532,:] = variable
        
    
    if not checker['right_hand_landmarks']:
        lm[:21,:] = lm[17+21]
    
    if not checker['left_hand_landmarks']:
        lm[532:,:] = lm[18+21]
    # print("this in cropped image")
    # for i in range(len(lm[21:54,:])):
    #       print("index: ",i,") pose: ",lm[21:54,:][i])
    
    cropped_image, (x_min, x_max, y_min, y_max) = self.detect_and_crop_person(frame, lm, margin=self.margin)
    if cropped_image.size == 0:
        adjusted_landmarks = np.zeros_like(landmarks_np)
        output_image = torch.zeros(3, self.target_size[1], self.target_size[0])
        return output_image, adjusted_landmarks

    processed_img, scale, (x_offset, y_offset) = self.resize_and_pad(cropped_image, target_size=self.target_size)

    if self.normalized:
        h, w, _ = frame.shape
        x_coords = lm[:, 0] * w
        y_coords = lm[:, 1] * h
    else:
        x_coords = lm[:, 0]
        y_coords = lm[:, 1]

    adjusted_x = (x_coords - x_min) * scale + x_offset
    adjusted_y = (y_coords - y_min) * scale + y_offset

    target_w, target_h = self.target_size
    # out_of_bounds = (
    #     (adjusted_x < 0) | (adjusted_x >= target_w) |
    #     (adjusted_y < 0) | (adjusted_y >= target_h)
    # )
    # adjusted_x[out_of_bounds] = 0.0
    # adjusted_y[out_of_bounds] = 0.0

    if self.normalized:
        adjusted_x /= target_w
        adjusted_y /= target_h

    adjusted_landmarks = np.stack([adjusted_x, adjusted_y], axis=-1)

    if self.transform:
        processed_img = self.transform(processed_img)
    else:
        processed_img = torch.from_numpy(processed_img).permute(2, 0, 1).float()
    if self.debuging:
      self.cropped_image = processed_img
    return processed_img, adjusted_landmarks

  def resize_and_pad(self,image, target_size=(512, 512)):
      target_w, target_h = target_size
      h, w, _ = image.shape
      if h == 0 or w == 0:
          return np.zeros((target_h, target_w, 3), dtype=np.uint8), 1.0, (0, 0)
      padded_image = np.zeros((target_h, target_w, 3), dtype=np.uint8)
      scale = min(target_w / w, target_h / h)
      new_w, new_h = int(w * scale), int(h * scale)
      if new_w == 0 or new_h == 0:
          return padded_image, 1.0, (0, 0)
      resized_image = cv2.resize(image, (new_w, new_h))
      x_offset = (target_w - new_w) // 2
      y_offset = (target_h - new_h) // 2
      padded_image[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_image
      return padded_image, scale, (x_offset, y_offset)

  def detect_and_crop_person(self,frame, lm, margin=10):
      h, w, _ = frame.shape
      if self.normalized:
          x_coords = lm[:, 0] * w
          y_coords = lm[:, 1] * h
      else:
          x_coords = lm[:, 0]
          y_coords = lm[:, 1]
      x_min = int(max(np.min(x_coords) - margin, 0))
      x_max = int(min(np.max(x_coords) + margin, w))
      y_min = int(max(np.min(y_coords) - margin, 0))
      y_max = int(min(np.max(y_coords) + margin, h))
      if x_max <= x_min or y_max <= y_min:
          return frame, (0, w, 0, h)
      crop = frame[y_min:y_max, x_min:x_max]
      if crop.size == 0:
          return frame, (0, w, 0, h)
      return crop, (x_min, x_max, y_min, y_max)
  def showImage(self):
    print("this cropped image: ")
    image = self.cropped_image.permute(1,2,0).numpy().astype(dtype=np.uint16)
    plt.imshow(image)
    plt.show()

class FeaturesExtractorFromImage():
  def __init__(self,debuging):
    if debuging:
      self.info = {
          "left_hand_image":None,
          "right_hand_image":None,
          "vertical":None,
          "horizontal":None,
          "left_distance":None,
          "right_distance":None
      }

    self.debuging= debuging
    pass

  def extractFeatures(self,image, landmarks_checker, scaled_landmarks):

    if not landmarks_checker["pose_landmarks"] and not landmarks_checker["right_hand_landmarks"] and not landmarks_checker["left_hand_landmarks"] and not landmarks_checker["face_landmarks"]:
      return list(np.zeros((39514)))
    scaled_landmarks = scaled_landmarks
    # right hands landmarks:  21
    # pose landmarks:  33
    # face landmarks:  478
    # left hands landmarks:  21
    right_hand_landmarks,pose_landmarks,face_landmarks,left_hand_landmarks = scaled_landmarks[:21,:],scaled_landmarks[21:54,:],scaled_landmarks[54:532,:],scaled_landmarks[532:,:]
    # print("this now in mediapipe")
    # for i in range(len(scaled_landmarks[21:54,:])):
    #       print("index: ",i,") pose: ",scaled_landmarks[21:54,:][i])
    
    vertical_regoins, horizontal_regions, right_hands_box_cordinations, pose_box_cordinations, face_box_cordinations, left_hands_box_cordinations =self.handsRegoins(
      landmarks_checker,
      right_hand_landmarks,
      pose_landmarks,
      face_landmarks,
      left_hand_landmarks,
      image
      )

    left_distance,right_distance,left_hand_image, right_hand_image = self.getHandsDistanceFromFace(
      landmarks_checker,
      right_hands_box_cordinations,
      pose_box_cordinations,
      face_box_cordinations,
      left_hands_box_cordinations,
      image
      )

    LR_hand_regions = np.array([vertical_regoins,horizontal_regions])

    # print(left_hand_image.reshape(-1).shape)
    # print(right_hand_image.reshape(-1).shape)
    # print(LR_hand_regions.reshape(-1).shape)
    # print(left_distance.reshape(-1).shape)
    # print(right_distance.reshape(-1).shape)
    # print(right_hand_landmarks.reshape(-1).shape)
    # print(pose_landmarks.reshape(-1).shape)
    # print(face_landmarks.reshape(-1).shape)
    # print(left_hand_landmarks.reshape(-1).shape)

    # print("maximum inside extract functino: ",left_hand_image.max())
    # plt.imshow( left_hand_image)
    # plt.show()


    frame_features = np.concatenate([left_hand_image.reshape(-1),right_hand_image.reshape(-1),LR_hand_regions.reshape(-1),left_distance.reshape(-1),right_distance.reshape(-1), right_hand_landmarks.reshape(-1),pose_landmarks.reshape(-1),face_landmarks.reshape(-1),left_hand_landmarks.reshape(-1)])
    return torch.tensor(frame_features)

  def getHandsDistanceFromFace(self,landmarks_checker,right_hands_box_cordinations, pose_box_cordinations, face_box_cordinations, left_hands_box_cordinations, image):
      margin = 10

      left_hand_image= image[left_hands_box_cordinations[0][1]-margin:left_hands_box_cordinations[1][1]+margin,
                        left_hands_box_cordinations[0][0]-margin:left_hands_box_cordinations[1][0]+margin]

      right_hand_image= image[right_hands_box_cordinations[0][1]-margin:right_hands_box_cordinations[1][1]+margin,
                        right_hands_box_cordinations[0][0]-margin:right_hands_box_cordinations[1][0]+margin]
      left_hand_image = cv2.resize(left_hand_image, (80,80)) if left_hand_image.shape[0] > 0 and left_hand_image.shape[1] > 0 else np.zeros((80,80,3))
      right_hand_image = cv2.resize(right_hand_image, (80,80)) if right_hand_image.shape[0] > 0 and right_hand_image.shape[1] > 0 else np.zeros((80,80,3))
      distance_between_left_face = np.array([np.abs(face_box_cordinations[0][0]-left_hands_box_cordinations[0][0]),np.abs(face_box_cordinations[1][1] -left_hands_box_cordinations[1][1])])
      distance_between_right_face = np.array([np.abs(face_box_cordinations[0][0]-right_hands_box_cordinations[0][0]),np.abs(face_box_cordinations[1][1] -right_hands_box_cordinations[1][1])])
      if self.debuging:
        self.info['left_distance'] = distance_between_left_face
        self.info['right_distance'] = distance_between_right_face
        self.info['left_hand_image'] = left_hand_image
        self.info['right_hand_image'] = right_hand_image
      return distance_between_left_face,distance_between_right_face,(left_hand_image),(right_hand_image)


  def handsRegoins(self, landmarks_checker, right_hand_landmarks,pose_landmarks,face_landmarks,left_hand_landmarks,image):
    right_hands_box_cordinations = np.min(right_hand_landmarks,axis = 0).astype(int), np.max(right_hand_landmarks,axis=0).astype(int)#min bottom left - max top right
    pose_box_cordinations = np.min(pose_landmarks,axis = 0).astype(int), np.max(pose_landmarks,axis=0).astype(int)#min bottom left - max top right
    face_box_cordinations = np.min(face_landmarks,axis = 0).astype(int), np.max(face_landmarks,axis=0).astype(int)#min bottom left - max top right
    left_hands_box_cordinations = np.min(left_hand_landmarks,axis = 0).astype(int), np.max(left_hand_landmarks,axis=0).astype(int)#min bottom left - max top right

    if not landmarks_checker['face_landmarks']:
      new_box_cordinations = np.array([pose_landmarks[8],pose_landmarks[7],pose_landmarks[10],pose_landmarks[9]])
      face_box_cordinations = np.min(new_box_cordinations,axis = 0).astype(int),np.max(new_box_cordinations,axis=0).astype(int)
        
    if not landmarks_checker["right_hand_landmarks"]:
      new_box_cordinations = np.array([pose_landmarks[21],pose_landmarks[19],pose_landmarks[15],pose_landmarks[17]])
      right_hands_box_cordinations = np.min(new_box_cordinations,axis = 0).astype(int),np.max(new_box_cordinations,axis=0).astype(int)
    if not landmarks_checker["left_hand_landmarks"]:
      new_box_cordinations = np.array([pose_landmarks[20],pose_landmarks[22],pose_landmarks[18],pose_landmarks[16]])
      left_hands_box_cordinations = np.min(new_box_cordinations,axis = 0).astype(int),np.max(new_box_cordinations,axis=0).astype(int)

    vertical_hands_locations =[]
    for hand in [left_hands_box_cordinations,right_hands_box_cordinations]:
      if hand[0][1] <=face_box_cordinations[0][1]:
        vertical_hands_locations.append(1)
      elif hand[0][1] >=face_box_cordinations[0][1] and hand[0][1] < pose_box_cordinations[0][1]:
        vertical_hands_locations.append(2)
      elif hand[0][1] <face_box_cordinations[1][1] and hand[0][1] >= pose_box_cordinations[0][1]:
        vertical_hands_locations.append(3)
      elif hand[0][1] >pose_box_cordinations[0][1] and hand[0][1]>= face_box_cordinations[1][1] and hand[0][1]< (pose_box_cordinations[0][1]/2)+pose_box_cordinations[0][1] :
        vertical_hands_locations.append(4)
      else:
        vertical_hands_locations.append(5)
    if self.debuging:
      print("this in mediapipe")
      m = image.copy()
      top_left, bottom_right = left_hands_box_cordinations
      cx = int((top_left[0] + bottom_right[0]) / 2)
      cy = int((top_left[1] + bottom_right[1]) / 2)
      center = (cx, cy)
      cv2.circle(m, center, radius=5, color=(0, 0, 255), thickness=-1)
      cv2.rectangle(m, tuple(face_box_cordinations[0]), tuple(face_box_cordinations[1]), (255, 0, 0), 2)
      cv2.rectangle(m, tuple(left_hands_box_cordinations[0]), tuple(left_hands_box_cordinations[1]), (255, 0, 255), 2)
      cv2.rectangle(m, tuple(right_hands_box_cordinations[0]), tuple(right_hands_box_cordinations[1]), (255, 255, 0), 2)
      cv2.rectangle(m, tuple(pose_box_cordinations[0]), tuple(pose_box_cordinations[1]), (255, 255, 255), 2)
      print("this is tuple: ",tuple(pose_box_cordinations[0]), tuple(pose_box_cordinations[1]))
      # for i in range(len(pose_landmarks)):
      #     print("index: ",i,") pose: ",pose_landmarks[i])
      plt.imshow(m)
      plt.show()


    horizontal_hands_locations = []
    for hand in [left_hands_box_cordinations,right_hands_box_cordinations]:
      if hand[0][0]< pose_box_cordinations[0][0]:
        horizontal_hands_locations.append(1)
      elif hand[0][0] >= pose_box_cordinations[0][0] and hand[0][0] < face_box_cordinations[0][0]:
        horizontal_hands_locations.append(2)
      elif hand[0][0] >= face_box_cordinations[0][0] and hand[0][0] < face_box_cordinations[1][0]:
        horizontal_hands_locations.append(3)
      elif hand[0][0] >=face_box_cordinations[1][0] and hand[0][0]< pose_box_cordinations[1][0]:
        horizontal_hands_locations.append(4)
      else:
        horizontal_hands_locations.append(5)
    if self.debuging:
      self.info['horizontal'] = horizontal_hands_locations
      self.info['vertical'] = vertical_hands_locations
    return vertical_hands_locations,horizontal_hands_locations, right_hands_box_cordinations, pose_box_cordinations, face_box_cordinations, left_hands_box_cordinations

  def showFeatures(self):
    print("left hand distance from the face: ", self.info['left_distance'])
    print("right hand distance from the face: ", self.info['right_distance'])
    print("vertical hands regions: ",self.info["vertical"])
    print("horizontal hands regions: ",self.info["horizontal"])
    print("left hand image: ")
    plt.imshow(self.info["left_hand_image"])
    plt.show()
    print("right hand image: ")
    plt.imshow(self.info["right_hand_image"])
    plt.show()

class MediapipeFeaturesExtractor():
  def __init__(self,debuging =True,target_size = (512,512)):      
    self.imageCropper = ImageCropper(debuging,target_size )
    self.featuresExtractorFromImage = FeaturesExtractorFromImage(debuging)
    self.debuging = debuging
    

  def extractFeaturesFromVideo(self, frames, landmarks, segmented_frames):
    cropped_images = []
    mediapipeFeatures = []
    previous_scaled_up_right_hand_landmarks = [0]
    previous_scaled_up_pose_landmarks = [0]
    previous_scaled_up_face_landmarks = [0]
    previous_scaled_up_left_hand_landmarks = [0]

    # for frame in tqdm(frames):
    #   self.extractor.extractLandmarks(frame)
    #   checkers.append(self.extractor.check())
    #   fframes.append(self.extractor.image)
    #   landmarks.append(self.extractor.get_x_y_landmarks_scaled_up())
    for frame,landmark,segmented_frame in zip(frames,landmarks, segmented_frames):
      
      checker = {
          "right_hand_landmarks":(landmark[-2][0]==1),
          "left_hand_landmarks":(landmark[-1][1]==1),
          "pose_landmarks":(landmark[-2][1]==1),
          "face_landmarks":(landmark[-1][0]==1),
      }
      if not checker['right_hand_landmarks'] and np.sum(previous_scaled_up_right_hand_landmarks) != 0 :
        landmark[:21,:] = previous_scaled_up_right_hand_landmarks

      if not checker['left_hand_landmarks'] and np.sum(previous_scaled_up_right_hand_landmarks) != 0 :
        landmark[532:-2,:] = previous_scaled_up_left_hand_landmarks

      if not checker['pose_landmarks'] and np.sum(previous_scaled_up_pose_landmarks) != 0:
        landmark[21:54,:]= previous_scaled_up_pose_landmarks
      if not checker['face_landmarks'] and np.sum(previous_scaled_up_face_landmarks) != 0:
        landmark[54:532,:]= previous_scaled_up_face_landmarks
      
      previous_scaled_up_right_hand_landmarks = landmark[:21,:]
      previous_scaled_up_pose_landmarks = landmark[21:54,:]
      previous_scaled_up_face_landmarks = landmark[54:532,:]
      previous_scaled_up_left_hand_landmarks = landmark[532:-2,:]
      
      cropped_image, _ = self.imageCropper.cropImage(segmented_frame, landmark[:-2], checker)
      new_frame, adjusted_landmark = self.imageCropper.cropImage(frame, landmark[:-2], checker)
      mediapipeFeature = self.featuresExtractorFromImage.extractFeatures(new_frame.permute(1,2,0).numpy().astype(dtype=np.uint16),checker,adjusted_landmark)
      cropped_images.append(cropped_image)
      mediapipeFeatures.append(mediapipeFeature)
      if self.debuging:
        self.showImages()
    return torch.stack(cropped_images), torch.stack(mediapipeFeatures)

  def showImages(self):
    if self.debuging:
      self.imageCropper.showImage()
      self.featuresExtractorFromImage.showFeatures()
    else:
      print("sett debuging flag to True")


In [3]:
# video = torch.load("/kaggle/input/where-gloss1/signer0_sample1026/video.pt",weights_only=False)
# landmarks = torch.load("/kaggle/input/where-gloss1/signer0_sample1026/landmarks.pt",weights_only=False)
# segmented_video = torch.load("/kaggle/input/where-gloss1/signer0_sample1026/segmented_video.pt",weights_only=False)

In [4]:
# plt.imshow(video[0]/255)
# plt.show()

In [5]:
# mediapipeFeaturesExtractor = MediapipeFeaturesExtractor(debuging=True,target_size=(512,512))
# cropped_images, mediapipeFeatures = mediapipeFeaturesExtractor.extractFeaturesFromVideo(np.array(video),np.array(landmarks),np.array(segmented_video))

In [6]:
# for i in mediapipeFeatures:
#     plt.imshow(i[0:19200].reshape(80,80,3)/255)
#     plt.show()
# train_dataset[0][0].shape

In [7]:
# for cropped_image in cropped_images.permute(0,2,3,1)/255:
#     plt.imshow(cropped_image)
#     plt.show()

In [8]:
class VideoDataset(Dataset):
  def __init__(self,videos_pathes,labels ,agumentation=False,temp_agumentation=False,RGB_only = False,getCropedImageToo= False , cropedtargetsize=(512, 512)):
    
    videos,labels = videos_pathes, labels
   
    if RGB_only:
        new_videos = []
        new_labels = []
        for index in range(len(videos)):
            # print(videos[index],videos[index].split('/')[-1].split('_')[-1])
            if videos[index].split('/')[-1].split('_')[-1].strip() == 'color.pt':
                new_videos.append(videos[index])
                new_labels.append(labels[index])
        videos = new_videos
        labels = new_labels
   
      
    self.videos = videos
    labels = labels

    unique_labels = list(set(labels))
    unique_labels.sort()
    label_to_index = {label: index for index, label in enumerate(unique_labels)}
    self.label_to_index = label_to_index
    labels = torch.tensor([label_to_index[label] for label in labels])
    one_hot_encoded_labels = F.one_hot(labels, num_classes=len(unique_labels))
    self.labels = one_hot_encoded_labels
    self.agumentation = agumentation
    self.temp_agumentation = temp_agumentation
    self.cropedtargetsize = cropedtargetsize
    self.getCropedImageToo =getCropedImageToo

    self.mediapipeFeaturesExtractor = MediapipeFeaturesExtractor(debuging=False,target_size=(512,512))

  def __len__(self):
    return len(self.videos)
  
    
  def __getitem__(self,index):    
      landmarks = torch.load(self.videos[index][0],weights_only=False)
      video = torch.load(self.videos[index][1],weights_only=False)
      segmented_video = torch.load(self.videos[index][2],weights_only=False)

      landmarks = np.array(landmarks)
      segmented_video = np.array(segmented_video)
      video = np.array(video)
      
      if self.agumentation:
          randomStarting = torch.randint(0, int(len(video)/2), (1,)).item()
      else:
          randomStarting = 0
      
      indices = torch.linspace(randomStarting, len(video)-1, steps=32).long()
      
      segmented_video = segmented_video[indices]
      landmarks = landmarks[indices]
      video = video[indices]
      if self.agumentation:
          video, segmented_video, agumented_landmarks = self.apply_consistent_augmentation(video,segmented_video, landmarks[:,:-2,:])
          landmarks[:,:-2,:]= agumented_landmarks
      cropped_images, mediapipeFeatures = self.mediapipeFeaturesExtractor.extractFeaturesFromVideo(video, landmarks, segmented_video)
      
      return mediapipeFeatures, cropped_images, self.labels[index]
      

  def apply_consistent_augmentation(self,frames,segmented_frames, landmarks):
    # Generate consistent augmentation parameters for all frames
    flip = random.random() > 0.5
    angle = random.uniform(-0.1, 0.1)  # Degrees
    brightness = random.uniform(0.50, 1.20)
    contrast = random.uniform(0.8, 1.2)
    zoom_factor = random.uniform(1, 1.2)

    augmented_frames = []
    augmented_landmarks = []
    agumented_segmented_frames =[]

    
    for frame, segmented_frame, frame_landmarks in zip(frames,segmented_frames, landmarks):
        
        # Convert frame to PIL Image
        frame_pil = Image.fromarray(frame)
        segmented_frame_pil = Image.fromarray(segmented_frame)
        width, height = frame_pil.size  # Original size

        # Ensure landmarks are a float32 tensor
        if not isinstance(frame_landmarks, torch.Tensor):
            frame_landmarks = torch.tensor(frame_landmarks, dtype=torch.float32)
        else:
            frame_landmarks = frame_landmarks.to(dtype=torch.float32)

        # # --- Step 1: Horizontal Flip ---
        # if flip:
        #     frame_pil = frame_pil.transpose(Image.FLIP_LEFT_RIGHT)
        #     frame_landmarks[:, 0] = width - frame_landmarks[:, 0]  # Flip x-coordinates
        #     segmented_frame_pil = segmented_frame_pil.transpose(Image.FLIP_LEFT_RIGHT)

        # --- Step 2: Rotation ---
        if angle != 0:
            # Rotate image (expand=False to maintain original size)
            frame_pil = frame_pil.rotate(angle, resample=Image.BILINEAR, expand=False)
            segmented_frame_pil = segmented_frame_pil.rotate(angle, resample=Image.BILINEAR, expand=False)
            # Rotate landmarks around the image center
            center = torch.tensor([width / 2, height / 2], dtype=torch.float32)
            angle_rad = torch.deg2rad(torch.tensor(angle, dtype=torch.float32))
            cos_theta = torch.cos(angle_rad)
            sin_theta = torch.sin(angle_rad)

            # Rotation matrix
            rotation_matrix = torch.tensor([
                [cos_theta, -sin_theta],
                [sin_theta, cos_theta]
            ], dtype=torch.float32)

            # Apply rotation: (landmarks - center) @ R + center
            frame_landmarks = (frame_landmarks - center) @ rotation_matrix + center

        # --- Step 3: Brightness/Contrast (no landmark change) ---
        frame_pil = transforms.functional.adjust_brightness(frame_pil, brightness)
        frame_pil = transforms.functional.adjust_contrast(frame_pil, contrast)
        
        segmented_frame_pil = transforms.functional.adjust_brightness(segmented_frame_pil, brightness)
        segmented_frame_pil = transforms.functional.adjust_contrast(segmented_frame_pil, contrast)

        # --- Step 4: Zoom & Crop ---
        # Calculate new dimensions and offsets (as floats)
        new_width = width * zoom_factor
        new_height = height * zoom_factor
        left = (new_width - width) / 2  # Float precision
        top = (new_height - height) / 2

        # Resize image (using integer dimensions)
        new_width_int = int(round(new_width))
        new_height_int = int(round(new_height))
        frame_pil = frame_pil.resize((new_width_int, new_height_int), Image.BILINEAR)

        # Crop to original size (using integer offsets)
        left_int = int(round(left))
        top_int = int(round(top))
        frame_pil = frame_pil.crop((left_int, top_int, left_int + width, top_int + height))
        segmented_frame_pil = segmented_frame_pil.crop((left_int, top_int, left_int + width, top_int + height))

        # Adjust landmarks (using precise float values)
        frame_landmarks = (frame_landmarks * zoom_factor) - torch.tensor([left, top], dtype=torch.float32)

        # --- Finalize ---
        agumented_segmented_frames.append(segmented_frame_pil)
        augmented_frames.append(frame_pil)
        augmented_landmarks.append(frame_landmarks)
    
    return np.array(augmented_frames).astype(dtype=np.uint16),np.array(agumented_segmented_frames).astype(dtype=np.uint16), np.array(augmented_landmarks)

  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
str("magdi").endswith("i")

True

In [10]:

CHECKPOINT_PATH = "/kaggle/input/attention-based-multi-module-fusion/data"
BASE_PATH = '/kaggle/input'
m = [
    "wait",
    "bring",
    "friend",
    "good",
    "get_well",
    "get-well",
    "thanks",
    "who",
    "where",
    "why",
    "time",
    "tomorrow",
    "always",
    "apologize",
    "goodbye",
    "single",
    "same",
    "hurry",
    "belt",
    "congratulation",
    "police",
    "single",
    "accident",
    "bed",
    "breakfast",
    "forbidden",
    "sibling",
    "angle",
    "glove",
    "full"
        ]
tester = []
train_video_pathes = []
train_labels = []
for gloss in os.listdir(BASE_PATH):
  if not gloss.__contains__("segmented") :
      continue
  glossName = os.listdir(os.path.join(BASE_PATH,gloss))[0]
  if glossName in m:
    for video_path in os.listdir(os.path.join(BASE_PATH,gloss,glossName)):
        files = os.listdir(os.path.join(BASE_PATH,gloss,glossName,video_path))
        tester.append(video_path)
        train_video_pathes.append([os.path.join(BASE_PATH,gloss,glossName,video_path,files[0]),os.path.join(BASE_PATH,gloss,glossName,video_path,files[1]),os.path.join(BASE_PATH,gloss,glossName,video_path,files[2])])
        train_labels.append(glossName.lower())

BASE_PATH = '/kaggle/input'
for gloss in os.listdir(BASE_PATH):
  if not gloss.endswith("-gloss") or not gloss.split("-gloss")[0] in m :  
      continue
  glossName = gloss.split("-")[0]   
  for video_path in os.listdir(os.path.join(BASE_PATH,gloss)):
    if video_path.endswith("json"):
        continue
    files = os.listdir(os.path.join(BASE_PATH,gloss,video_path))
    tester.append(video_path)
    train_video_pathes.append([os.path.join(BASE_PATH,gloss,video_path,files[0]),os.path.join(BASE_PATH,gloss,video_path,files[1]),os.path.join(BASE_PATH,gloss,video_path,files[2])])
    train_labels.append(glossName.lower())

test_video_pathes = []
test_labels = []
for t in range(1,6):
    BASE_PATH = f'/kaggle/input/test-segmented-continue-part{t}-db/gloss'
    for gloss in os.listdir(BASE_PATH):
      if gloss in m:
        for video_path in os.listdir(os.path.join(BASE_PATH,gloss)):
            files = os.listdir(os.path.join(BASE_PATH,gloss,video_path))
            test_video_pathes.append([os.path.join(BASE_PATH,gloss,video_path,files[0]),os.path.join(BASE_PATH,gloss,video_path,files[1]),os.path.join(BASE_PATH,gloss,video_path,files[2])])
            test_labels.append(gloss.lower())
len(train_video_pathes)

# BASE_PATH = "/kaggle/input/test-continue-part2-db/output"
# for gloss in os.listdir(BASE_PATH):
#   if gloss in m:
#     for video_path in os.listdir(os.path.join(BASE_PATH,gloss)):
#         files = os.listdir(os.path.join(BASE_PATH,gloss,video_path))
#         test_video_pathes.append([os.path.join(BASE_PATH,gloss,video_path,files[0]),os.path.join(BASE_PATH,gloss,video_path,files[1])])
#         test_labels.append(gloss.lower())
len(train_video_pathes)


3243

In [11]:
len(set(train_labels)),len(set(test_labels))

(26, 26)

In [12]:
print(len(tester),len(set(tester)),len(test_labels))
train_dictanory = {t:0 for t in set(np.sort(train_labels))}
test_dictanory = {t:0 for t in set(np.sort(test_labels))}
for t in train_labels:
    train_dictanory[t]+=1
    
for t in test_labels:
    test_dictanory[t]+=1

train_dictanory,test_dictanory

3243 3243 179


({'bed': 127,
  'single': 126,
  'thanks': 126,
  'accident': 125,
  'get_well': 124,
  'tomorrow': 117,
  'hurry': 125,
  'time': 123,
  'apologize': 127,
  'goodbye': 127,
  'breakfast': 125,
  'where': 127,
  'who': 126,
  'belt': 127,
  'wait': 125,
  'glove': 122,
  'why': 127,
  'forbidden': 123,
  'same': 120,
  'full': 127,
  'friend': 121,
  'police': 126,
  'always': 123,
  'bring': 124,
  'sibling': 126,
  'good': 127},
 {'bed': 8,
  'single': 7,
  'thanks': 8,
  'accident': 5,
  'get_well': 6,
  'tomorrow': 7,
  'hurry': 6,
  'time': 7,
  'apologize': 6,
  'goodbye': 7,
  'breakfast': 7,
  'where': 8,
  'who': 8,
  'belt': 6,
  'wait': 6,
  'glove': 7,
  'why': 6,
  'forbidden': 7,
  'same': 7,
  'full': 8,
  'friend': 6,
  'police': 7,
  'always': 8,
  'bring': 8,
  'sibling': 7,
  'good': 6})

In [13]:
train_dataset = VideoDataset(
    train_video_pathes,
   train_labels,
  agumentation=True,temp_agumentation=False,getCropedImageToo=True
    )
train_dataloader = DataLoader(train_dataset,batch_size=2,shuffle=True,
    num_workers=8,  # Use 4 workers (Adjust based on CPU cores)
    pin_memory=True, # Speed up CPU → GPU transfer
    prefetch_factor=5,
                                       timeout=60
                              )
test_dataset = VideoDataset(
    test_video_pathes,
    test_labels,getCropedImageToo=True
    )
test_dataloader = DataLoader(test_dataset,batch_size=2,shuffle=True,
     num_workers=4,  # Use 4 workers (Adjust based on CPU cores)
    pin_memory=True, # Speed up CPU → GPU transfer
    prefetch_factor=5,
                                      timeout=60
                             )

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [14]:
# for i in train_dataset[0][0]:
#     print(i.sum())

In [15]:
# torch.load("/kaggle/input/who-segmented-continue-part1-db/who/signer0_sample1084/segmented_video.pt",weights_only=False)


In [16]:
# for i in test_dataset[0][1]:
#     plt.imshow(i.permute(1,2,0)/255)
#     plt.show()
# for i in test_dataset[0][0]:
#     plt.imshow(i[19200:19200*2].reshape(80,80,3)/255)
#     plt.show()
# train_dataset[0][0].shape
# train_dataset[0][0].shape

In [17]:
# mpfeatures, img,label =train_dataset[10]
# for i in range(32):
#     plt.imshow(img[i].permute(1,2,0))
#     plt.show()

In [18]:
# mpfeatures, img,label =train_dataset[0]
# for i in range(32):
#     plt.imshow(mpfeatures[i][:19200].reshape(80,80,3)/255)
#     plt.show()

In [19]:
# next(iter(train_dataloader))

In [20]:

def predict(model,test_loader):
    predictions = []
    actual = []
    model.eval()
    with torch.no_grad():
        for x,croped_image,y in tqdm(test_loader):    
            x = x.float().to(device) # batch_size, num_frames, channels, height, width
            croped_image = croped_image.float().to(device)
            outputs = model(x,croped_image)
            predictions.extend(outputs.cpu().detach().numpy() )
            actual.extend(y.cpu().detach().numpy())
    return predictions, actual


class PositionalEmbedding(nn.Module):
    def __init__(self):
        super(PositionalEmbedding, self).__init__()
        self.pos_embed = nn.Embedding(32, 2048)

    def forward(self, x):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, device=x.device).unsqueeze(0).repeat(x.size(0), 1)  
        pos_embed = self.pos_embed(pos) 
        return x + pos_embed



class CrossAttention(nn.Module):
    def __init__(self, image_dim, pose_dim, project_dim, num_heads=4):
        super(CrossAttention, self).__init__()
        self.image_pre_norm = nn.LayerNorm(normalized_shape=image_dim)
        self.pose_pre_norm = nn.LayerNorm(normalized_shape=pose_dim)

        self.pose_project = nn.Linear(pose_dim, project_dim)
        self.image_project = nn.Linear(image_dim, project_dim)

        self.image_self_attention= nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)
        self.pose_self_Attenstion = nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)
        
        self.image_layer_norm = nn.LayerNorm(normalized_shape=project_dim)
        self.pose_layer_norm = nn.LayerNorm(normalized_shape=project_dim)
        # self.output_layer_norm = nn.LayerNorm(normalized_shape=int(project_dim*2))


        self.cross_attention1 = nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)
        self.cross_attention2 = nn.MultiheadAttention(embed_dim=project_dim, num_heads=num_heads, batch_first=True)
        
        
    def forward(self, image_input, pose_input):
        image_features = self.image_pre_norm(image_input)
        pose_features = self.pose_pre_norm(pose_input)
        
        pose_projection = self.pose_project(pose_features)
        image_projection = self.image_project(image_features)

        pose_projection_attention, _ = self.pose_self_Attenstion(query=pose_projection, key=pose_projection, value=pose_projection)
        image_projection_attention, _ = self.image_self_attention(query=image_projection, key=image_projection, value=image_projection)

        residual_image_connection = image_projection+image_projection_attention
        residual_pose_connection = pose_projection+pose_projection_attention
        
        image = self.image_layer_norm(residual_image_connection)
        pose = self.pose_layer_norm(residual_pose_connection)
        
        attn_output1, _ = self.cross_attention1(query=image, key=pose, value=pose)
        attn_output2, _ = self.cross_attention2(query=pose, key=image, value=image)
        
        # output = self.final_feedforward_layer(torch.cat([ attn_output1, attn_output2 ],dim=2))

        # output = self.output_layer_norm(output)

        return torch.cat([ attn_output1, attn_output2 ],dim=2)  

class Model(nn.Module):
    def __init__(self,embedding = 5):
        super().__init__()

        self.crossAttentionLayer = CrossAttention(image_dim=6320, pose_dim=3102,project_dim=1024, num_heads=4)

        
        mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        mobilenet_features = mobilenet.features

        t=0
        for layer in mobilenet.parameters():
            t+=1

        for i,layer in enumerate(mobilenet.parameters()):
            layer.requires_grad = False
            if i== (t-4) or i== (t-5):
              layer.requires_grad = True
                
        self.cropedImageCNN = nn.Sequential(
                mobilenet_features,
               
                nn.Conv2d(1280, 1580, kernel_size=3),
                nn.BatchNorm2d(1580),
                nn.SiLU(),
                nn.AvgPool2d(kernel_size=2),
            
                nn.Conv2d(1580, 1580, kernel_size=3),
                nn.BatchNorm2d(1580),
                nn.SiLU(),
                nn.AvgPool2d(kernel_size=2),
                
                nn.Flatten()
            )

       
        self.leftHandCNN = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 40x40
        
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 20x20
        
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 10x10
        
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 5x5
            nn.Flatten(),
            nn.Linear(256 * 5 * 5, 1280),
            
        )

        self.rightHandCNN = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 40x40
        
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 20x20
        
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 10x10
        
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.SiLU(),
            nn.AvgPool2d(2),       # -> 5x5
            nn.Flatten(),
            nn.Linear(256 * 5 * 5, 1280),
            
        )

        self.leftHandVRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.leftHandHRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.rightHandVRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.rightHandHRegionEmbedding = nn.Sequential(
            nn.Linear(1,embedding),
        )
        self.leftHandDistance = nn.Sequential(
            nn.Linear(2,embedding),
        )
        self.rightHandDistance = nn.Sequential(
            nn.Linear(2,embedding),
        )
        self.landmarksEmbedding = nn.Sequential(
            nn.Linear(1106 ,512),            
        ) 

        encoder_layer = nn.TransformerEncoderLayer(
                d_model=2048,
                nhead=8,
                dim_feedforward=3072,
                dropout=0.0,
                activation='gelu',
                batch_first=True
            )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        
        self.positionalEmbedding = PositionalEmbedding()

        self.classifier = nn.Sequential(
            nn.Linear(2048, 26),
        )
        self.landmarksLayerNorm = nn.LayerNorm(normalized_shape=1106)
        # self.dropout1 = nn.Dropout(0.05)
        # self.dropout2 = nn.Dropout(0.05)
        
    def forward(self,x,croped_image, train= False):
        batch_size,frames, features = x.size()
        
        # print(x.size())
        
        # print(x.size())
            
        LHI,RHI,LHVR,RHVR,LHHR,RHHR,LHD,RHD,ALL_LANDMARKS =x[:,:,:19200]/255,x[:,:,19200:38400]/255,x[:,:,38400],x[:,:,38401],x[:,:,38402],x[:,:,38403],x[:,:,38404:38406],x[:,:,38406:38408],x[:,:,38408:]
        LHI = LHI.reshape(batch_size,frames,80,80,3).float()
        LHI = LHI.reshape(batch_size*frames, 80,80,3)
        LHI = LHI.permute(0,3,1,2)
        LHI_output = self.leftHandCNN(LHI)
        LHI_output = LHI_output.reshape(batch_size,frames,-1) #1280
        
        RHI = RHI.reshape(batch_size,frames,80,80,3).float()
        RHI = RHI.reshape(batch_size*frames, 80,80,3)
        RHI = RHI.permute(0,3,1,2)
        # print(RHI.shape)
        RHI_output = self.rightHandCNN(RHI)
        RHI_output = RHI_output.reshape(batch_size,frames,-1) #1280

        # print('left hand regoin',LHR.shape)
        LHVR_output = self.leftHandVRegionEmbedding(LHVR.unsqueeze(-1).float())
        LHHR_output = self.leftHandHRegionEmbedding(LHHR.unsqueeze(-1).float())
        RHVR_output = self.rightHandVRegionEmbedding(RHVR.unsqueeze(-1).float())
        RHHR_output = self.rightHandHRegionEmbedding(RHHR.unsqueeze(-1).float())

        LHD_output = self.leftHandDistance(LHD.float())
        RHD_output = self.rightHandDistance(RHD.float())

        all_landmarks_output = self.landmarksEmbedding(self.landmarksLayerNorm(ALL_LANDMARKS.float()))
        
        batch_size,frames,c,w,h = croped_image.shape
        croped_images_input = croped_image.reshape(batch_size*frames,c,w,h)

        croped_output = self.cropedImageCNN(croped_images_input/255)#[128, 512, 3, 3]

        croped_output = croped_output.reshape(batch_size,frames,-1)
        # croped_output = self.dropout1(croped_output)
        # print(croped_output.shape)
        flattend_output = torch.cat([LHI_output, RHI_output,LHVR_output,LHHR_output, RHVR_output,RHHR_output,LHD_output, RHD_output, all_landmarks_output],dim=2)
        # flattend_output = self.dropout2(flattend_output)
        # flattend_output = self.dropout(flattend_output)
        combined_output = self.crossAttentionLayer(croped_output,flattend_output)
        combined_output = self.positionalEmbedding(combined_output)
        
        transformer_output = self.transformer(combined_output)

        global_average_pooling = transformer_output.mean(dim=1)


        output = self.classifier(global_average_pooling)
        
        
        return output



{val:key for key,val in train_dataset.label_to_index.items()},{val:key for key,val in test_dataset.label_to_index.items()}

({0: 'accident',
  1: 'always',
  2: 'apologize',
  3: 'bed',
  4: 'belt',
  5: 'breakfast',
  6: 'bring',
  7: 'forbidden',
  8: 'friend',
  9: 'full',
  10: 'get_well',
  11: 'glove',
  12: 'good',
  13: 'goodbye',
  14: 'hurry',
  15: 'police',
  16: 'same',
  17: 'sibling',
  18: 'single',
  19: 'thanks',
  20: 'time',
  21: 'tomorrow',
  22: 'wait',
  23: 'where',
  24: 'who',
  25: 'why'},
 {0: 'accident',
  1: 'always',
  2: 'apologize',
  3: 'bed',
  4: 'belt',
  5: 'breakfast',
  6: 'bring',
  7: 'forbidden',
  8: 'friend',
  9: 'full',
  10: 'get_well',
  11: 'glove',
  12: 'good',
  13: 'goodbye',
  14: 'hurry',
  15: 'police',
  16: 'same',
  17: 'sibling',
  18: 'single',
  19: 'thanks',
  20: 'time',
  21: 'tomorrow',
  22: 'wait',
  23: 'where',
  24: 'who',
  25: 'why'})

In [21]:

def get_details(actual,predictions):
    actual_label = np.argmax(np.array(actual),axis=1)
    predicted_label = np.argmax(predictions, axis =1)
    accuracy = (actual_label == predicted_label).sum() / len(actual_label)
    number_of_classes = len(train_dataset.label_to_index)
    
    details = {
        'result_per_label': {i:0 for i in range(number_of_classes)},#how much i classiy for this class
        'total_per_label':  {i:0 for i in range(number_of_classes)}#how much the total that belong to this class
    }
    for i in range(len(actual_label)):
        result = 1 if actual_label[i] == predicted_label[i] else 0
        class_index = actual_label[i]
        details['result_per_label'][class_index]+= result
        details['total_per_label'][class_index]+= 1
    index_to_label = {val:key for key,val in train_dataset.label_to_index.items()}
    deta = {}
    for i in range(number_of_classes):
        if details['total_per_label'][i] ==0 or details['result_per_label'][i] ==0 :
            print('index: ',i,' Gloss: ',index_to_label[i],' accuracy: 0')
        else:
            print('index: ',i,' Gloss: ',index_to_label[i],' accuracy: ',(details['result_per_label'][i]/details['total_per_label'][i]))
            deta [i] = 'index: ' +str(i)+ ' Gloss: '+ str(index_to_label[i])+' accuracy: '+ str(details['result_per_label'][i]/details['total_per_label'][i])
    return accuracy, deta
    
def train(model,classification_loss_fn,epoch,train_loader,test_loader,scheduler,test_accuracy):
    max_seconds = 10.5*60*60  # 10 hours and 30 minutes
    start_time = time.time()
    if os.path.exists(os.path.join(CHECKPOINT_PATH,'checkpoint.pth')):
        checkpointsaver = torch.load(os.path.join(CHECKPOINT_PATH,'checkpoint.pth'), map_location='cpu')
        with open(os.path.join(CHECKPOINT_PATH,'data.json'), 'r') as file:
            metadatasaver = json.load(file)
    else:
        checkpointsaver = {}
        metadatasaver = {}

    total_loss = []
    glosses_details =[]
    current_best_accuracy = 0
    for ep in range(epoch,epoch+1000):
        model.train()
        total_loss_per_epoch = 0
        accuarcy = 0
        total_samples= 0
        for i, (x, croped_images, y) in enumerate(train_loader):
            elapsed = time.time() - start_time
            if elapsed > max_seconds:
                print(f"\n⏱️ Training loop exceeded 9 hours 30 minutes. Breaking at batch {i}.")
                return None

            optimizer.zero_grad()
        
            x = x.float().to(device)
            croped_images = croped_images.float().to(device)
        
            if y.ndim > 1:  # one-hot -> indices
                y = torch.argmax(y, dim=1)
        
            y = y.long().to(device)
        
            classification_output = model(x, croped_images)  # shape: [B, 2]
        
            # Compute loss
            classification_loss = classification_loss_fn(classification_output, y)
            loss = classification_loss
            total_loss_per_epoch += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
        
            # Compute accuracy
            predictions = torch.argmax(classification_output, dim=1)
            accuarcy += (predictions == y).sum().item()
            total_samples += y.size(0)
        
            print(f"\rEpoch {ep+1}, loss: {(total_loss_per_epoch/(i+1)):.8f}, [{i}/{len(train_loader)}]", end="")

        print(f"\rEpoch {ep+1}, loss: {(total_loss_per_epoch/len(train_loader)):.8f}")
        print()
        print("train accuracy is: ",(accuarcy/total_samples))
        avg_loss = total_loss_per_epoch/len(train_loader)
        total_loss.append(avg_loss)
        actual,predictions = predict(model,test_loader)
        acc, deta = get_details(actual,predictions)
        
        
        

        
        checkpointsaver['model_state_dict'] =  model.state_dict()
        checkpointsaver['optimizer_state_dict'] =  optimizer.state_dict()
        checkpointsaver['scheduler_state_dict'] =  scheduler.state_dict()
        checkpointsaver["epoch"] =  ep+1

        
         
        metadatasaver[ "epoch"]= ep
        metadatasaver[ "total train accuracy"]=(accuarcy/total_samples)
        metadatasaver[ "total test accuracy"]= acc
        metadatasaver["glosses test accuracy"]= deta
        

        
        if acc >= test_accuracy:
            test_accuracy =acc
            metadatasaver['best test accuracy'] = test_accuracy
            metadatasaver['best test accuracy details'] = deta
            checkpointsaver['best_model_state_dict']=model.state_dict()
            
        print('total accuracy: ',acc)
        with open("data.json", "w") as f:
            json.dump(metadatasaver, f,indent=4)
        print("best test accuracy: ",test_accuracy)


        torch.save(checkpointsaver, "checkpoint.pth")

        scheduler.step(avg_loss)
        print("\n///////////current learning rate: ",optimizer.param_groups[0]['lr']," /////////////\n")

    return total_loss
    

In [22]:
os.listdir(CHECKPOINT_PATH)

['data.json', 'checkpoint.pth']

In [23]:
if not os.path.exists(os.path.join(CHECKPOINT_PATH,'checkpoint.pth')):
    model = Model().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=8e-05)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',         
        factor=0.5,         
        patience=3,         
        verbose=True        
    )
    
    start_epoch= 0 
    test_accuracy = 0
else:
    import torch
    import gc
    from torch import nn
    
    # 1. Force clean GPU memory
    def clear_gpu_memory():
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()  # Reset memory tracking
    
    clear_gpu_memory()
    
    # 2. Load checkpoint differently
    def load_checkpoint_safely(checkpoint_path, device):
        # Load to CPU first
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        
        # Create model on CPU
        model = Model()  # Don't move to GPU yet
        with open(os.path.join(CHECKPOINT_PATH,'data.json'), 'r') as file:
            data = json.load(file)
            
        if 'best test accuracy' in data:
            test_accuracy = data["best test accuracy"]
        else:
            test_accuracy = 0
         # Load state dict in eval mode (uses less memory)
        model.eval()
        model.load_state_dict(checkpoint['best_model_state_dict'])
        
        # Now move to GPU
        model = model.to(device)
        
        # Create optimizer after model is on GPU
        optimizer = torch.optim.AdamW(model.parameters(), lr=8e-05)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        return {
            'model': model,
            'optimizer': optimizer,
            'epoch': checkpoint['epoch'],
            'scheduler_state': checkpoint.get('scheduler_state_dict', None)
        },test_accuracy
    

    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # Load checkpoint
    loaded, test_accuracy = load_checkpoint_safely(os.path.join(CHECKPOINT_PATH,'checkpoint.pth'), device)
    
    model = loaded['model']
    optimizer = loaded['optimizer']
    start_epoch = loaded['epoch']
    
    # Recreate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3,
        verbose=True
    )
    
    if loaded['scheduler_state'] is not None:
        scheduler.load_state_dict(loaded['scheduler_state'])
                
classification_loss = nn.CrossEntropyLoss ()



/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [24]:
print("started epoch: ", start_epoch)
print("test accuracy: ",test_accuracy)

started epoch:  82
test accuracy:  0.8938547486033519


In [25]:
# print("model classifier: ", model.classifier)

In [ ]:
history = train(model,classification_loss ,start_epoch,train_dataloader,test_dataloader,scheduler,test_accuracy)

Epoch 83, loss: 0.15415485, [1621/1622]

train accuracy is:  0.9759481961147086


100%|██████████| 90/90 [01:02<00:00,  1.44it/s]


index:  0  Gloss:  accident  accuracy:  1.0
index:  1  Gloss:  always  accuracy:  0.6363636363636364
index:  2  Gloss:  apologize  accuracy:  1.0
index:  3  Gloss:  bed  accuracy:  0.6153846153846154
index:  4  Gloss:  belt  accuracy:  1.0
index:  5  Gloss:  breakfast  accuracy:  1.0
index:  6  Gloss:  bring  accuracy:  0.7777777777777778
index:  7  Gloss:  forbidden  accuracy:  1.0
index:  8  Gloss:  friend  accuracy:  1.0
index:  9  Gloss:  full  accuracy:  1.0
index:  10  Gloss:  get_well  accuracy:  0.8333333333333334
index:  11  Gloss:  glove  accuracy:  0.8333333333333334
index:  12  Gloss:  good  accuracy:  1.0
index:  13  Gloss:  goodbye  accuracy:  0.8571428571428571
index:  14  Gloss:  hurry  accuracy:  0.8
index:  15  Gloss:  police  accuracy:  0.625
index:  16  Gloss:  same  accuracy:  0.625
index:  17  Gloss:  sibling  accuracy:  1.0
index:  18  Gloss:  single  accuracy:  0.875
index:  19  Gloss:  thanks  accuracy:  0.8
index:  20  Gloss:  time  accuracy:  1.0
index:  21  

100%|██████████| 90/90 [01:05<00:00,  1.38it/s]


index:  0  Gloss:  accident  accuracy:  1.0
index:  1  Gloss:  always  accuracy:  0.7272727272727273
index:  2  Gloss:  apologize  accuracy:  1.0
index:  3  Gloss:  bed  accuracy:  1.0
index:  4  Gloss:  belt  accuracy:  1.0
index:  5  Gloss:  breakfast  accuracy:  0.7777777777777778
index:  6  Gloss:  bring  accuracy:  0.7272727272727273
index:  7  Gloss:  forbidden  accuracy:  0.875
index:  8  Gloss:  friend  accuracy:  1.0
index:  9  Gloss:  full  accuracy:  0.75
index:  10  Gloss:  get_well  accuracy:  1.0
index:  11  Gloss:  glove  accuracy:  0.8333333333333334
index:  12  Gloss:  good  accuracy:  1.0
index:  13  Gloss:  goodbye  accuracy:  1.0
index:  14  Gloss:  hurry  accuracy:  0.75
index:  15  Gloss:  police  accuracy:  0.6363636363636364
index:  16  Gloss:  same  accuracy:  1.0
index:  17  Gloss:  sibling  accuracy:  0.7777777777777778
index:  18  Gloss:  single  accuracy:  0.7777777777777778
index:  19  Gloss:  thanks  accuracy:  0.8888888888888888
index:  20  Gloss:  time 